In [1]:
import utils
import pandas as pd
import difflib
from urllib.error import HTTPError

In [2]:
stadiums_path = 'C:\Tiger Graph\data\stadiums_20150302.csv'
matches_path = 'C:\Tiger Graph\data\games.csv'
team_id_path = 'C:\Tiger Graph\data\Teams.csv'

In [3]:
google_account = {'clientID':'nvYuSlmhDApA3TcC1g8tO', 'secretKey':'KRuncrb0CpNfPsA2p31sAf0ZV6u2EnkC4V3OiqGY'}
icloud_account = {'clientID':'dsvx3PTajVYN5X6lqT80q', 'secretKey':'JuZ0Qf3ymcyXuCFuHFnXLJFraVbgMhhXhPapOOSx'}
kubrick_account = {'clientID':'myWMloJRcG9SLFaSUcA8Y', 'secretKey':'S9Rjpii26EeWgx4E1BEsclpYpiutHiSvF1G9QCDg'}

In [4]:
stadiums = utils.get_stadium_locations(stadiums_path)
team_ids = utils.get_team_ids(team_id_path)

In [5]:
#makesure allteam names and thus dict keys match up
locations_to_get =[]
for id in team_ids:
    if team_ids[id] in stadiums.keys():
        pass
    else:
        for i in stadiums.keys():
            if team_ids[id] in i:
                team_ids[id] = i
        if len(difflib.get_close_matches(team_ids[id],stadiums.keys()))==0:
            locations_to_get.append(team_ids[id])

In [6]:
#all names and keys need to have values and match up and shit
stads = set(stadiums.keys())
teams = set(team_ids.values())
stadiums_without_team_ids = stads.difference(teams)
teams_without_stadiums = teams.difference(stads)

In [7]:
new_ids=utils.create_new_ids(list(team_ids.keys()),list(stadiums_without_team_ids))

In [8]:
for i,j in zip(new_ids,stadiums_without_team_ids):
   team_ids[i]=j

stads = set(stadiums.keys())
teams = set(team_ids.values())
stadiums_without_team_ids = stads.difference(teams)
teams_without_stadiums = teams.difference(stads)

In [9]:
try:
    assert len(list(teams_without_stadiums)) == 0
except AssertionError:
    print('There is a stadium without coordinates.')

In [10]:
try:
    assert len(stadiums_without_team_ids) == 0
except AssertionError:
    print('There is a team that has not been assigned to a stadium.')

In [11]:
matches = utils.get_match_dates(matches_path,team_ids)

In [12]:
counter = 0
data = {}
for match in matches:
	try:
		json = utils.api_request(matches[match],stadiums,google_account['clientID'],google_account['secretKey'])
		weather = utils.get_weather(json,matches[match]['date'])
		data[match]=weather
		counter += 1
	except HTTPError:
		break

In [13]:
remainder = {match:matches[match] for match in list(matches)[counter-1:]}

In [14]:
for match in remainder:
	try:
		json = utils.api_request(matches[match],stadiums,icloud_account['clientID'],icloud_account['secretKey'])
		weather = utils.get_weather(json,matches[match]['date'])
		data[match]=weather
		counter += 1
	except HTTPError:
		break

In [15]:
remainder = {match:matches[match] for match in list(matches)[counter-1:]}
for match in remainder:
	try:
		json = utils.api_request(matches[match],stadiums,kubrick_account['clientID'],kubrick_account['secretKey'])
		weather = utils.get_weather(json,matches[match]['date'])
		data[match]=weather
		counter += 1
	except HTTPError:
		break

In [24]:
df = pd.DataFrame.from_dict(data, orient='index')

In [25]:
display(df)

,tempC,dewpointC,humidity,windSpeedKPH,weather,feelslikeC,message
81,22.72,12.58,53.0,5.91,Partly Cloudy,22.72,NaN
82,24.96,14.95,54.0,6.48,Mostly Sunny,24.96,NaN
83,16.19,14.62,90.0,9.85,Partly Cloudy,16.19,NaN
84,23.67,13.08,51.0,11.62,Partly Cloudy,23.67,NaN
85,21.11,8.29,44.0,21.32,Mostly Sunny,21.11,NaN
...,...,...,...,...,...,...,...
5310,NaN,NaN,NaN,NaN,NaN,NaN,Error
5311,NaN,NaN,NaN,NaN,NaN,NaN,Error
5381,NaN,NaN,NaN,NaN,NaN,NaN,Error
5382,NaN,NaN,NaN,NaN,NaN,NaN,Error


In [26]:
df.to_csv('C:\Tiger Graph\data\match_weather_data.csv')